## Before you start:
**Tools → Settings → Editor → completions / suggestions / linting → disable**

**Task 1:** Study how basic reference counting works
Create a string and investigate its reference count at different stages. Return a tuple: (count_after_creation, count_after_ref, count_after_del)

In [1]:
import sys

def task1():
    string_test = "new test string"
    count_after_creation = sys.getrefcount(string_test)
    string_ref = string_test
    count_after_ref = sys.getrefcount(string_test)
    del string_ref
    count_after_del = sys.getrefcount(string_test)
    return (count_after_creation, count_after_ref, count_after_del)

# Check
print("Task 1:", task1())

Task 1: (4, 5, 4)


**Task 2:** Function impact on Reference Counting
Create a list and pass it to a function.Compare reference count before and after function call.Return a tuple: (count_before_call, count_during_call, count_after_call)

In [2]:
def task2():
    test_list = [1, 2, 3]
    count_before_call = sys.getrefcount(test_list)
    def process_list(lst):
        return sys.getrefcount(lst)

    count_during_call = process_list(test_list)
    count_after_call = sys.getrefcount(test_list)
    return (count_before_call, count_during_call, count_after_call)

# Check
print("Task 2:", task2())

Task 2: (2, 3, 2)


**Task 3:** Cyclic references and Memory Leaks.
Create two objects with a cyclic reference, then break it.Use gc to check the number of collected objects.Return the number of objects collected by gc after breaking the reference.

In [3]:
import gc

def task3():
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None

    gc.disable()
    gc.collect()

    node_a = Node('a')
    node_b = Node('b')
    node_a.ref = node_b
    node_b.ref = node_a
    del node_a
    del node_b
    collected_objects = gc.collect()

    gc.enable()
    return collected_objects

print("Task 3:", task3())

Task 3: 2


**Task 4:** Comparing Reference Count for different data types
Compare reference count for numbers, strings, lists and dictionaries.Return a dictionary with reference count for each type after creation.

In [5]:
def task4():
    test_number0 = 50505
    test_string0 = "testtsts"
    test_list0 = [1, 2, 3]
    test_dict0 = {'a': 1, 'b': 2}

    return sys.getrefcount(test_number0), sys.getrefcount(test_string0),\
    sys.getrefcount(test_list0), sys.getrefcount(test_dict0)


print("Task 4:", task4())

Task 4: (4, 3, 2, 2)


**Task 5:** Weak References
Create two objects with a weak reference between them.Ensure the objects can be deleted by the garbage collector.Return True if the weakref does not increase reference count.

In [6]:
import sys
import weakref
import gc

def task5():
    class Data:
        def __init__(self, value):
            self.value = value

    abc = Data(25)
    count_before = sys.getrefcount(abc)

    abc_weak = weakref.ref(abc)
    count_after = sys.getrefcount(abc)

    del abc
    gc.collect()

    abc_deleted = (abc_weak() is None)
    return count_before == count_after and abc_deleted
print("Task 5:", task5())

Task 5: True


**Task 6:** Monitoring the Garbage Collector
Register a callback function to track GC activity.Return a list of events tracked by the callback.

In [8]:
def task6():
  events = []
  def callback(phase, info):
      events.append((phase, info))

  def create_garbage():
      a = [1]
      b = [2]
      a.append(b)
      b.append(a)

  gc.callbacks.append(callback)
  create_garbage()
  gc.collect()
  gc.callbacks.remove(callback)

  return events

print("Task 6:", task6())

Task 6: [('start', {'generation': 2, 'collected': 0, 'uncollectable': 0}), ('stop', {'generation': 2, 'collected': 2, 'uncollectable': 0})]


**Task 7:** GC Generation Analysis
Create objects and trace their movement between GC generations.Return a tuple with the number of objects in each generation before and after object creation.

In [16]:
import gc

def task7():
    gc.collect()
    before = gc.get_count()

    a_7 = [1, 2]
    b_7 = [3, 4]
    a_7.append(b_7)
    b_7.append(a_7)

    gc.collect()
    after = gc.get_count()

    return before, after


print("Task 7:", task7())

Task 7: ((20, 0, 0), (4, 0, 0))


**Task 8:** Monitoring Garbage Collection Thresholds
Study how GC generation counters change when creating objects.Return a dictionary with the state of the counters before and after object creation.

In [20]:
import gc

def task8():
    gc.collect()
    initial_threshold = gc.get_threshold()
    initial_count = gc.get_count()

    objs = [object() for _ in range(100000)]

    after_creation_count = gc.get_count()
    collected = gc.collect(0)
    after_collect_count = gc.get_count()

    return {
        'thresholds': initial_threshold,
        'initial_count': initial_count,
        'after_creation': after_creation_count,
        'after_collect_gen0': after_collect_count,
        'collected_objects': collected
    }


print("Task 8:", task8())

Task 8: {'thresholds': (700, 10, 10), 'initial_count': (18, 0, 0), 'after_creation': (15, 0, 0), 'after_collect_gen0': (1, 1, 0), 'collected_objects': 0}


**Task 9:** Quick GC check
Check if the GC collects cyclic references.Return the difference in the number of objects before and after collection.

In [30]:
import gc

def task9():
    gc.collect()
    a, b, c = [], [], []
    a.append(b); b.append(c); c.append(a)

    count_before = len(gc.get_objects())
    gc.collect()
    count_after = len(gc.get_objects())

    return count_before - count_after

print("Task 9:", task9())

Task 9: 35


**Task 10:** Detector for "undying" objects
Find objects that survive forced garbage collection.Output the number of such objects.

In [31]:
import gc

def task10():
    gc.collect()
    gc.set_debug(gc.DEBUG_SAVEALL)
    class Persistent:
        def __del__(self):
            pass

    a_10 = Persistent()
    b_10 = Persistent()
    a_10.ref = b_10
    b_10.ref = a_10
    del a_10, b_10

    gc.collect()
    result = len(gc.garbage)
    gc.set_debug(0)
    del gc.garbage[:]
    return result

print("Task 10:", task10())

Task 10: 2
